In [1]:
import pandas as pd
import re
import json
import math
from itertools import permutations, combinations
from IPython.display import clear_output
from time import sleep
from abc import ABC, abstractmethod, abstractproperty

from grid_builder import *
from grid import *
from tile import *
from tiles import *
from destructable import *
from unit import *
from command import *
from ability import *
from shell import *
from beam import *
from game import *
from executor import *
from dfs import *
from dfs_viewer import *
from main import *
from melee import *

In [2]:
# MOVE (7, 3) to (5, 5)
# MOVE (3, 2) to (3, 4)
# BOULDER SHELL at (5, 1)
# MOVE (5, 3) to (7, 3)
# REGULAR SHELL at (7, 1)
# CLUSTER SHELL at (3, 2)

In [3]:
# Game(reset_grid(), True)

In [4]:
# grid = reset_grid3()

In [5]:
# grid.end_commands[4]

In [6]:
# Game(reset_grid2(), True)

In [7]:
grid = reset_grid3()
display(grid.show())
dfs = DFS(grid, 8)

,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,🟢,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,🟢,Artillery Mech ♡♡♡♡♡,💦,🟢
3,🟢,🟢,Siege Mech ♡♡♡♡♡,🟢,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡♡♡♡♡,🟢,🏘️ ϟϟ,💦
6,💦,🗻,Alpha Scarab ♡♡♡♡,Beetle ♡♡♡♡,🟢,🟢,Scarab ♡♡,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


In [8]:
v = DFSViewer(reset_grid3(), dfs)

In [9]:
for i in range(10):
    v.show(i)

569


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,🟢,🟢,🟢,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,Beetle ♡♡♡,Boulder Mech ♡♡♡♡,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
6,💦,🟢,🟢,🟢,Alpha Firefly ♡,🟢,🟢,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


{'Power': 14, 'Veks': 2, 'Vek Total Health': 4, 'Mechs': 3, 'Mech Total Health': 13}
569


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,🟢,🟢,🟢,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,Beetle ♡♡♡,Boulder Mech ♡♡♡♡,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
6,💦,🟢,🟢,🟢,Alpha Firefly ♡,🟢,🟢,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


{'Power': 14, 'Veks': 2, 'Vek Total Health': 4, 'Mechs': 3, 'Mech Total Health': 13}
569


...

{'Power': 14, 'Veks': 2, 'Vek Total Health': 7, 'Mechs': 3, 'Mech Total Health': 14}
567


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,Boulder Mech ♡♡♡♡♡,🟢,🟢,🟢,🟢,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,Beetle ♡♡♡,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
6,💦,🟢,🟢,🟢,Alpha Firefly ♡♡♡♡,🟢,🟢,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


{'Power': 14, 'Veks': 2, 'Vek Total Health': 7, 'Mechs': 3, 'Mech Total Health': 14}
567


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🗻,🗻
1,💦,Boulder Mech ♡♡♡♡♡,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,🟢,🟢,🟢,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,Beetle ♡♡♡,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
6,💦,🟢,🟢,🟢,Alpha Firefly ♡♡♡♡,🟢,🟢,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


{'Power': 14, 'Veks': 2, 'Vek Total Health': 7, 'Mechs': 3, 'Mech Total Health': 14}


In [10]:
v.show(0, True)

,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,🟢,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,🟢,Artillery Mech ♡♡♡♡♡,💦,🟢
3,🟢,🟢,Siege Mech ♡♡♡♡♡,🟢,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡♡♡♡♡,🟢,🏘️ ϟϟ,💦
6,💦,🗻,Alpha Scarab ♡♡♡♡,Beetle ♡♡♡♡,🟢,🟢,Scarab ♡♡,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


MOVE (3, 2) to (3, 3)


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,🟢,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,🟢,Artillery Mech ♡♡♡♡♡,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡♡♡♡♡,🟢,🏘️ ϟϟ,💦
6,💦,🗻,Alpha Scarab ♡♡♡♡,Beetle ♡♡♡♡,🟢,🟢,Scarab ♡♡,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


CLUSTER SHELL at (6, 3)


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,🟢,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,Boulder Mech ♡♡♡♡♡,🟢,Artillery Mech ♡♡♡♡♡,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,Alpha Firefly ♡♡♡♡♡,🟢,🏘️ ϟϟ,💦
6,💦,🗻,Alpha Scarab ♡,Beetle ♡♡♡♡,🟢,🟢,Scarab ♡♡,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


...

BOULDER SHELL at (6, 4)


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,🟢,🟢,🟢,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,🟢,Boulder Mech ♡♡♡♡,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
6,💦,🗻,🟢,Beetle ♡♡♡,Alpha Firefly ♡♡,🟢,Scarab ♡♡,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


569


,0,1,2,3,4,5,6,7
0,💦,💦,🗻,🏘️ ϟϟ,Artillery Mech ♡♡♡♡♡,🟢,🗻,🗻
1,💦,🟢,🏘️ ϟϟ,🏘️ ϟϟ,🟢,🟢,🗻,🗻
2,🟢,🟢,🟢,🟢,🟢,🟢,💦,🟢
3,🟢,🟢,🟢,Siege Mech ♡♡♡♡,💦,💦,💦,🟢
4,🗻,🏘️ ϟϟ,🟢,Beetle ♡♡♡,Boulder Mech ♡♡♡♡,🟢,🏘️ ϟϟ,💦
5,🟢,🏘️ ϟϟ,🟢,🟢,🟢,🟢,🏘️ ϟϟ,💦
6,💦,🟢,🟢,🟢,Alpha Firefly ♡,🟢,🟢,🟢
7,💦,🟢,🟢,🟢,🟢,🟢,🟢,🟢


{'Power': 14, 'Veks': 2, 'Vek Total Health': 4, 'Mechs': 3, 'Mech Total Health': 13}


In [11]:
grid = reset_grid3()





In [12]:
history = {}
for string, action in moves:
    name = repr(action)
    if string == "DO" and name in history:
        print(f"NOPE {name}")
    if string == "DO":
        history[name] = True

NameError: name 'moves' is not defined

In [ ]:
sdf = {}

In [ ]:
pip = ["Stuff", "Thing", sdf]

In [ ]:
del sdf

In [ ]:
pip

In [ ]:
pip.pop(2)

In [ ]:
pip


In [ ]:
dfs = DFS(grid, 6)
moves = dfs.protected[0]

def try_moves(moves):
    grid = reset_grid3()
    history = {}
    frontier = DFSViewer.gen_frontier(grid)
    for string, action in moves:
        name = repr(action)
        if string == "DO":
            com = frontier[name]
            com.execute()
            score = DFS.rate(grid)
            history[id(action)] = (com, score)
        else:
            com, score = history[id(action)]
            com.undo()
            new_score = DFS.rate(grid)
            if new_score != score:
                print(f"{new_score} does not equal {score} after undoing {command} with id {id(action)}")
                break

        frontier = DFSViewer.gen_frontier(grid)

In [ ]:
history[2132117014792]

In [ ]:
id(grid)

In [ ]:
grid = reset_grid3()
v = DFSViewer(grid, dfs)

In [ ]:
v.show(1, True)

In [ ]:
stop

In [ ]:
# grid.show()

In [ ]:
# MOVE (3, 2) to (1, 2)
# CLUSTER SHELL at (6, 2)
# MOVE (5, 3) to (6, 1)
# REGULAR SHELL at (6, 6)
# MOVE (7, 3) to (7, 2)
# BOULDER SHELL at (4, 2)

In [ ]:
grid = reset_grid()
DFSViewer(grid, dfs)

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
stop

In [ ]:
DFS.rate(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
stop

In [ ]:
grid = reset_grid()

In [ ]:
grid.show()

In [ ]:
DFS.rate_dict(grid)

In [ ]:
dfs.show()

In [ ]:
dfs.explored

In [ ]:
grid = reset_grid()
Game(grid)

In [ ]:
sm.ge

In [ ]:
choice.command.__dict__.copy()

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
ex = Executor()

In [ ]:
ex.execute(siege_mech.gen_actions()[2])

In [ ]:
grid.show()

In [ ]:
ex.execute(siege_mech.gen_actions()[4])

In [ ]:
grid.show()

In [ ]:
ex

In [ ]:
ex.undo()

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
ex = Executor()

score = DFS.rate(grid)

DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
ex.execute(frontier[0][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[1][0])
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

ex.undo()
score = DFS.rate(grid)

frontier = DFS.gen_rated_frontier(grid)
frontier.sort(key=lambda x:x[1], reverse=True)
frontier

In [ ]:
print(score)

In [ ]:
ex.execute(frontier[2][0])
score = DFS.rate(grid)

show = grid.show()

dic = DFS.rate_dict(grid, True)

frontier = DFS.gen_rated_frontier(grid)
score = DFS.rate(grid)

frontier.sort(key=lambda x:x[1], reverse=True)

score = DFS.rate(grid)

ex.execute(frontier[0][0])
score = DFS.rate(grid)

dic = DFS.rate_dict(grid, True)

show = grid.show()

In [ ]:
print(score)

In [ ]:
ex.undo()
score = DFS.rate(grid)

show = grid.show()

ex.execute(frontier[1][0])
score = DFS.rate(grid)

grid.show()

ex.undo()
score = DFS.rate(grid)

grid.show()

ex.execute(frontier[2][0])
score = DFS.rate(grid)

grid.show()

grid.get_tile((3,6)).visitor

grid.get_tile((5,5)).visitor

grid.get_tile((4,6)).type_object.health

grid.get_tile((4,5)).type_object.health

DFS.rate_dict(grid, True)

grid.show()

In [ ]:
print(score)

new score 42 != old score 37 at 
MOVE (7, 3) to (5, 5)
MOVE (3, 2) to (3, 4)
CLUSTER SHELL at (3, 2)
BOULDER SHELL at (5, 1)
MOVE (5, 3) to (3, 3)
HEAL +1

[MOVE (7, 3) to (5, 5), MOVE (3, 2) to (3, 4), CLUSTER SHELL at (3, 2), BOULDER SHELL at (5, 1), MOVE (5, 3) to (7, 3), HEAL +1, 'UNDO', REGULAR SHELL at (7, 1), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 2), HEAL +1, 'UNDO', REGULAR SHELL at (6, 2), 'UNDO', 'UNDO', MOVE (5, 3) to (3, 3), REGULAR SHELL at (3, 6), 'UNDO', REGULAR SHELL at (3, 7), 'UNDO', HEAL +1]

In [ ]:
stop

In [ ]:
stop

In [ ]:
dfs = DFS(grid, 5)

In [ ]:
dfs.show()

In [ ]:
stop

In [ ]:
grid.show()

In [ ]:
frontier = DFS.gen_rated_frontier(grid)

In [ ]:
frontier

In [ ]:
dfs = DFS(grid)

In [ ]:
DFS.rate_dict(grid)

In [ ]:
grid.show()

In [ ]:
stop

In [ ]:
[x for x in map(lambda x:x*5, [1,2,3,4,5,6])]

In [ ]:
grid.show()

In [ ]:
DFS.rate(grid)

In [ ]:
grid.show()

In [ ]:
DFS.gen_frontier(grid)

In [ ]:
def playback(code, grid):
    history = []
    for char in code:
        num = int(char)
        frontier = DFS.gen_frontier(grid)
        act = frontier[num]
        act.execute()
        history.append(act)
    return history
    

In [ ]:
grid.show()

In [ ]:
dfs = DFS(grid)

In [ ]:
ls = [(i,v) for i,v in dfs.explored.items()]
ls.sort(key=lambda iv: dfs.get_score_single(iv[1]), reverse=True)
ls    

In [ ]:
class Perm:
    def __init__(self):
        self.options = ["Move", "Shoot"]
        self.ls = []
        self.perms = []
        self.recurse()
        
    def recurse(self):
        frontier = self.gen_frontier()
        if not frontier:
            self.perms.append(self.ls)
            print(self.ls)
            return
            
        for option in frontier:
            self.ls.append(option)
            self.recurse()
            self.ls.pop()
        
    def gen_frontier(self):
        frontier = []
        move_count = self.ls.count("Move")
        shoot_count = self.ls.count("Shoot")
        if move_count < 3:
            frontier.append("Move")
        
        if move_count > shoot_count and shoot_count < 3:
            frontier.append("Shoot")
            
        return frontier
        
perm = Perm()  

In [ ]:
class Perm2:
    def __init__(self):
        self.shoots = ["Shoot1","Shoot2","Shoot3"]
        self.moves = ["Move1", "Move2", "Move3"]
        self.combos = []
        self.combo_1()
        self.combo_2()
        
    def __repr__(self):
        string = ""
        for i in self.combos:
            string += repr(i) +'\n'
        return string
            
    def last_move(self,tup):
        for move in self.moves:
            if move not in tup:
                return move,
            
            
    def generate(self, r=3):
        for shoot in permutations(shoots):
            for move in combinations(moves, r):
                yield move, shoot
                
    def combo_1(self):
        for move,shoot in self.generate(3):
            self.combos.append(move+shoot)
            
            
    def combo_2(self):
        for move,shoot in self.generate(2):
            move += self.last_move(move)
            self.combos.append((move[0],move[1],shoot[0],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],move[1],shoot[0],shoot[1],move[2],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],move[2],shoot[1],shoot[2]))
            self.combos.append((move[0],shoot[0],move[1],shoot[1],move[2],shoot[2]))
            
            
    def add_combos_3(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=3):
                self.combos.append(move+shoot)
                
                
    def add_combos_2(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                move += self.last_move(move)
                
            
            
    def stuff(self):
        for shoot in permutations(shoots):
            for move in combinations(moves, r=2):
                print((x[0],x[1],y[0],y[1],last_elem(x),y[1]))